In [1]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()
groq_api_key = os.getenv('GROQ_API_KEY')

if not groq_api_key:
    raise ValueError("Groq API key not found in .env file")

In [2]:
import os
import pickle
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq

# Initialize the HuggingFace Embedding Model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

c:\Users\zubay\.conda\envs\fyp\Lib\importlib\__init__.py:90: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)
C:\Users\zubay\AppData\Local\Temp\ipykernel_23540\3748620440.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install

In [3]:
# Load documents from a text file and add metadata
def load_documents(file_path):
    loader = TextLoader(file_path)
    documents = loader.load()
    
    # Add metadata to each document (e.g., file name)
    for doc in documents:
        doc.metadata["source"] = file_path
    return documents


In [4]:

# Split documents into chunks
def split_documents(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    split_docs = text_splitter.split_documents(documents)
    return split_docs

In [5]:
# Create embeddings and handle storage
def embed_documents(split_docs, embedding_model):
    EMBEDDINGS_FOLDER = "embeddings"
    EMBEDDINGS_FILE = os.path.join(EMBEDDINGS_FOLDER, "emb01.pkl")

    if not os.path.exists(EMBEDDINGS_FOLDER):
        os.makedirs(EMBEDDINGS_FOLDER)

    if os.path.exists(EMBEDDINGS_FILE):
        print(f"Loading existing embeddings from {EMBEDDINGS_FILE}...")
        with open(EMBEDDINGS_FILE, 'rb') as f:
            embedded_docs = pickle.load(f)
    else:
        print("Creating new embeddings...")
        texts = [doc.page_content for doc in split_docs]
        embedded_docs = embedding_model.embed_documents(texts)

        with open(EMBEDDINGS_FILE, 'wb') as f:
            pickle.dump(embedded_docs, f)

    return embedded_docs


In [6]:
# Store embeddings in Chroma vector store
def store_embeddings(split_docs, embedding_model):
    vector_store = Chroma.from_documents(split_docs, embedding_model) 
    return vector_store

In [7]:
# Build the RAG pipeline
def build_rag_pipeline(vector_store):
    retriever = vector_store.as_retriever()
    return retriever


In [8]:

# Initialize the LLM
def initialize_llm():
    llm = ChatGroq(
        model="llama-3.1-70b-versatile",
        temperature=0.2,  # Lower temperature for more consistent evaluations
    )
    return llm

In [9]:
# Querying the retriever and LLM
def query_llm(llm, retriever, query):
    # Retrieve relevant documents
    results = retriever.get_relevant_documents(query)

    # Use the LLM to process the retrieved documents
    if results:
        # Combine results for the LLM prompt, and track their sources
        context = "\n".join([doc.page_content for doc in results])
        prompt = f"""Please consider information from 9th to 12th-grade textbooks. If the provided context is relevant to the student's query, use it to give a precise answer. Otherwise, answer based on your own knowledge, considering the material covered in books for these grades. Don't mention in response that the context is not relevant. You can simply say "I cannot find relevant data from your book but I will explain you the general concept about" and so on. At the end, encourage student to ask conversational questions related to the topics in the book based on the query.
        
        Context:
        {context}
        Student Query:
        {query}
        """
        response = llm.invoke(prompt)
        
        # Capture the actual text chunks used
        relevant_texts = [doc.page_content for doc in results]
        
        # Return both the response and relevant texts
        return response, relevant_texts
    else:
        return "No relevant documents found.", []


In [10]:
# Main execution flow
if __name__ == "__main__":
    # Load your document
    documents = load_documents('resources/9thComputerScience_updated.txt')
    split_docs = split_documents(documents)
    embedded_docs = embed_documents(split_docs, embedding_model)
    vector_store = store_embeddings(split_docs, embedding_model)
    retriever = build_rag_pipeline(vector_store)
    
    # Initialize the LLM
    llm = initialize_llm()


Loading existing embeddings from embeddings\emb01.pkl...


In [11]:
# Example query 
query = "explain boolean proposition"
response, relevant_texts = query_llm(llm, retriever, query)

# Output response and relevant text chunks
print(response.content)
print("==============================================")
print("\nRelevant text chunks used in the response:")
for text in relevant_texts:
    print("Chunk: ==============================")
    print(text[:300])

C:\Users\zubay\AppData\Local\Temp\ipykernel_23540\32019177.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


A Boolean proposition is a statement that can be either true or false. It is a fundamental concept in Boolean algebra and is used to represent logical statements. A proposition can be a simple statement, such as "It is raining" or a compound statement, such as "It is raining and today is Sunday".

In Boolean algebra, propositions are represented using letters, such as P, Q, and R. Each proposition has a truth value, which is either true (T) or false (F). The truth value of a proposition depends on the truthfulness or falsity of the statement.

For example, consider the proposition P = "Islamabad is the capital of Pakistan". This proposition is true because Islamabad is indeed the capital of Pakistan. On the other hand, the proposition Q = "The sun rises in the west" is false because the sun rises in the east.

Propositions can be combined using logical operators, such as AND, OR, and NOT. The truth value of a compound proposition depends on the truth values of the individual propositio

In [12]:
for text in relevant_texts:
    print("=====================================")
    print(text)

operator is shown in Table 2-11.
Truth Table for complex Boolean expressions: We can make truth table for any combination of these operators. For NOT(P) Q NOT(P)AND Q
example, if we need to make a truth table - of “It is not raining and today is Sunday”.
It means the proposition NOT(P) AND Q.
P T T F The truth table for this compound Table 2-12 proposition is shown Table 2-12.
2.5.5 Laws of Boolean Algebra The laws of Boolean Algebra help us to simplify complex Boolean expressions. Some laws are discussed in the following.
e Commutative Law Commutative Law states that the order of application of two separate propositions is not important. So,
a) A.B=B8B.A (The order in which two variables are AND’ed makes no difference.)
b) A+B=B+A _ (The order in which two variables are OR’ed makes no difference.)
We can use truth tables (Table 2-13a, Table 2-13b) to verify this law for AND and OR operations respectively.
Table 2-13a Table 2-13b
Do you know?
True and False are called Boolean values. T

In [14]:
# Example query 
query = "who gave the idea of boolean values and on what date"
response, relevant_texts = query_llm(llm, retriever, query)

# Output response and relevant text chunks
print(response.content)
print("==============================================")
print("\nRelevant text chunks used in the response:")
for text in relevant_texts:
    print("Chunk: ==============================")
    print(text[:300])

The idea of Boolean values was given by George Boole in his book "The Laws of Thought". George Boole was born on 2 November 1815.

Relevant text chunks used in the response:
Chunk: ==============================
Do you know?
True and False are called Boolean values. The idea was given by George Boole (2 November 1815 —8 December 1864) in his book “The Laws of Thought”.
2.5.2 Truth Values
Every proposition takes one of two values true or false, and these values are called the truth values. Truth value is giv
Chunk: ==============================
operator is shown in Table 2-11.
Truth Table for complex Boolean expressions: We can make truth table for any combination of these operators. For NOT(P) Q NOT(P)AND Q
example, if we need to make a truth table - of “It is not raining and today is Sunday”.
It means the proposition NOT(P) AND Q.
P T T 
Chunk: ==============================
A network of networks


Computer Science — 9



Truth value A value that is either true or false

Verification

In [15]:
# Example query 
query = "how ip4 and ip6 works"
response, relevant_texts = query_llm(llm, retriever, query)

# Output response and relevant text chunks
print(response.content)
print("==============================================")
print("\nRelevant text chunks used in the response:")
for text in relevant_texts:
    print("Chunk: ==============================")
    print(text[:300])

I'll explain how IPv4 and IPv6 work.

IPv4 (Internet Protocol Version 4) is the older standard of IP addressing. It consists of 32 bits, divided into four groups separated by dots (.). Each group can contain a decimal value from 0 to 255. Since each group can have a maximum value of 255, which is equivalent to 11111111 in binary, 8 bits are required to represent each group. Therefore, a total of 32 bits (4 groups x 8 bits per group) are needed to store an IPv4 address.

On the other hand, IPv6 (Internet Protocol Version 6) is the newer standard, designed to accommodate the increasing number of devices connecting to the internet. It consists of 128 bits, divided into eight groups separated by colons (:). Each group can contain four hexadecimal digits. Since each hexadecimal digit requires 4 bits, a group of four hexadecimal digits requires 16 bits. Therefore, a total of 128 bits (8 groups x 16 bits per group) are needed to store an IPv6 address.

Here's a comparison of IPv4 and IPv6:

I